### Examples Of Pyspark ML

#### INICIAMOS SESION

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('dataframe').getOrCreate()

CARGAMOS CSV

training = spark.read.csv('data/test1.csv', header=True, inferSchema=True)

In [2]:
training = spark.read.csv('data/test1.csv', header=True, inferSchema=True)

In [3]:
training.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



En PySpark se trabaja de forma diferente. Tendremos que agrupar nuestras variables independientes de forma que queden todas en una columna y dentro de una lista, por lo que crearemos un vector de ensamblaje o "vector assembler", de tal modo que queden así esas variables independientes:
- [Age, Experience]

Lo que haremos con estas dos, será tratarlas como una nueva variable independiente:
- [Age, Experience] ----> nueva_variable_independiente

In [4]:
from pyspark.ml.feature import VectorAssembler 

In [5]:
feature_assembler = VectorAssembler(inputCols=['age', 'Experience'], 
                                    outputCol='Independent features') # 1:46:27 del vídeo

In [6]:
output = feature_assembler.transform(training)

Veremos que se crea una nueva columna cuyos valores se corresponden a unos array con el contenido de aquellas variables independientes que hemos agrupado. Esto será nuestro input feature o lo que solíamos definir como train.

In [7]:
output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



Seleccionamos las columnas que nos interesan para nuestro modelo: el train (Independent Features) y el test (Salary)

In [8]:
finalized_data = output.select('Independent features', 'Salary')
finalized_data.show()

+--------------------+------+
|Independent features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



A continuación, entrenaremos un modelo de regresión lineal.

In [9]:
from pyspark.ml.regression import LinearRegression

In [17]:
(train, test) = finalized_data.randomSplit([0.75, 0.25])
regressor = LinearRegression(featuresCol='Independent features', labelCol='Salary')
regressor = regressor.fit(train)

In [18]:
regressor.coefficients

DenseVector([736.8421, 736.8421])

In [19]:
regressor.intercept

-175.43859649151398

In [20]:
prediction = regressor.evaluate(test)

In [21]:
prediction.predictions.show()

+--------------------+------+------------------+
|Independent features|Salary|        prediction|
+--------------------+------+------------------+
|          [21.0,1.0]| 15000|16035.087719298195|
|          [29.0,4.0]| 20000| 24140.35087719302|
|          [30.0,8.0]| 25000| 27824.56140350881|
+--------------------+------+------------------+



In [22]:
# Errores

prediction.meanAbsoluteError, prediction.meanSquaredError

(2666.6666666666756, 8730686.365035541)

In [24]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import pandas as pd
df = pd.read_csv('data/test1.csv')


In [30]:
df

,Name,age,Experience,Salary
0,Krish,31,10,30000
1,Sudhanshu,30,8,25000
2,Sunny,29,4,20000
3,Paul,24,3,20000
4,Harsha,21,1,15000
5,Shubham,23,2,18000


In [48]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(["Salary", "Name"], axis=1), 
                                                    df["Salary"], 
                                                    train_size=0.75)

In [49]:
train, test = train_test_split(df.drop(["Name"], axis=1), train_size=0.75)

In [50]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [53]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
mean_absolute_error(y_test, lr.predict(X_test))

8333.333333333336

In [52]:
mean_squared_error(y_test, lr.predict(X_test))

74138888.88888893

In [54]:
lr.coef_

array([ 2666.66666667, -2000.        ])

In [55]:
lr.intercept_

-38833.33333333334

In [57]:
from sklearn.datasets import load_iris
iris = load_iris()

In [65]:
dt = pd.DataFrame(data=iris["data"], columns=iris["feature_names"])
#dt.columns = iris["feature_names"]
dt["target"] = iris["target"]
dt

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [68]:
dt.to_csv("iris.csv")
pyspark_iris = spark.read.csv("iris.csv", header=True, inferSchema=True)

In [71]:
from pyspark.ml.classification import LogisticRegression

In [72]:
feature_assembler = VectorAssembler(inputCols=iris["feature_names"], 
                                    outputCol='Independent features') # 1:46:27 del vídeo

In [74]:
output = feature_assembler.transform(pyspark_iris)
output.show()

+---+-----------------+----------------+-----------------+----------------+------+--------------------+
|_c0|sepal length (cm)|sepal width (cm)|petal length (cm)|petal width (cm)|target|Independent features|
+---+-----------------+----------------+-----------------+----------------+------+--------------------+
|  0|              5.1|             3.5|              1.4|             0.2|     0|   [5.1,3.5,1.4,0.2]|
|  1|              4.9|             3.0|              1.4|             0.2|     0|   [4.9,3.0,1.4,0.2]|
|  2|              4.7|             3.2|              1.3|             0.2|     0|   [4.7,3.2,1.3,0.2]|
|  3|              4.6|             3.1|              1.5|             0.2|     0|   [4.6,3.1,1.5,0.2]|
|  4|              5.0|             3.6|              1.4|             0.2|     0|   [5.0,3.6,1.4,0.2]|
|  5|              5.4|             3.9|              1.7|             0.4|     0|   [5.4,3.9,1.7,0.4]|
|  6|              4.6|             3.4|              1.4|      

In [79]:
finalized_data = output.select('Independent features', 'target')
(train, test) = finalized_data.randomSplit([0.75, 0.25])
regressor = LogisticRegression(featuresCol='Independent features', labelCol='target')
regressor = regressor.fit(train)

In [84]:
prediction = regressor.evaluate(test)

In [85]:
prediction.predictions.show()

+--------------------+------+--------------------+--------------------+----------+
|Independent features|target|       rawPrediction|         probability|prediction|
+--------------------+------+--------------------+--------------------+----------+
|   [4.4,3.0,1.3,0.2]|     0|[49.5376095262499...|[0.99999999974033...|       0.0|
|   [4.4,3.2,1.3,0.2]|     0|[51.9507114781452...|[0.99999999998521...|       0.0|
|   [4.6,3.6,1.0,0.2]|     0|[57.1251968136341...|[0.99999999999991...|       0.0|
|   [4.7,3.2,1.3,0.2]|     0|[48.9219838466421...|[0.99999999955255...|       0.0|
|   [4.7,3.2,1.6,0.2]|     0|[46.5545506606085...|[0.99999999766802...|       0.0|
|   [4.8,3.0,1.4,0.3]|     0|[43.1060989921099...|[0.99999989715820...|       0.0|
|   [4.8,3.4,1.9,0.2]|     0|[45.5906435493025...|[0.99999999784345...|       0.0|
|   [4.9,2.5,4.5,1.7]|     2|[-10.856584842221...|[3.93363310933212...|       1.0|
|   [4.9,3.0,1.4,0.2]|     0|[43.7005857450668...|[0.99999986769441...|       0.0|
|   

In [86]:
prediction.accuracy

0.9428571428571428

In [87]:
prediction.recallByLabel

[1.0, 0.9090909090909091, 0.875]

In [88]:
prediction.weightedRecall

0.9428571428571428